<a href="https://colab.research.google.com/github/BernardBernardes/colab_pipeline_integrations/blob/main/bdgd2opendss_colab_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚀 Clonagem do Repositório e Instalação do Pacote

Nesta etapa inicial, realizamos a preparação do ambiente para utilizar o pacote `bdgd2opendss`. O que acontece aqui:

1. **Clonamos o repositório** diretamente do GitHub para o ambiente do Colab.
2. **Acessamos a pasta** onde está o arquivo `setup.py`, necessário para instalação.
3. **Instalamos o pacote em modo editável** (`pip install -e .`), permitindo modificar o código e ver as alterações sem reinstalar.
4. Por fim, **voltamos para a pasta padrão do Colab** (`/content/`) para manter a organização do workspace.

Esse passo é essencial para garantir que todos os módulos e funções do pacote fiquem disponíveis para uso nas próximas células.

In [ ]:
# Clonar o repositório
!git clone https://github.com/pauloradatz/bdgd2opendss.git

# Mudar para o diretório onde está o setup.py
%cd /content/bdgd2opendss/
# Instalar os requisitos corretamente
!pip install -e .

# Volta para o diretório de trabalho padrão do Colab (opcional, mas boa prática)
%cd /content/

## 📁 Upload de Arquivo ZIP

Agora é hora de enviar os dados para o ambiente do Colab!

Com o comando `files.upload()`, será exibido um botão para selecionar e enviar arquivos diretamente do seu computador. Neste caso, espera-se que você envie um **arquivo `.zip`** contendo os dados do projeto a serem processados.

🔧 O arquivo enviado será armazenado automaticamente na pasta padrão do Colab: `/content/`.

➡️ Após o upload, o script usará esse arquivo para extração e processamento nas próximas etapas.

In [ ]:
from google.colab import files
uploaded = files.upload()

## 📦 Extração do Arquivo ZIP e Identificação da Pasta

Com o arquivo `.zip` já carregado, agora seguimos com:

1. **Identificação do nome do arquivo enviado**, para acessar seu caminho completo.
2. **Extração do conteúdo** do `.zip` diretamente na pasta `/content/` do Colab.
3. **Leitura da estrutura interna do `.zip`**, identificando a **pasta raiz criada** dentro dele (caso exista).
4. Armazenamos o caminho da pasta extraída na variável `pasta_extraida_path`, que será utilizada nas etapas seguintes do processamento.

🧠 Essa lógica permite que o código funcione mesmo que o nome da pasta dentro do `.zip` varie entre execuções, tornando o processo mais robusto e automático.

In [ ]:
import zipfile
import os

# Passo 2: Obter o nome do arquivo enviado
zip_filename = next(iter(uploaded))  # Pega o primeiro (e único) arquivo enviado
zip_path = f"/content/{zip_filename}"

# Passo 3: Extrair o zip para /content/
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

# Passo 4: Identificar a pasta criada
# Obtemos a lista de itens extraídos que estão no zip
conteudo_zip = zip_ref.namelist()

# Pegamos o primeiro diretório presente no .zip
# (caso o zip crie uma pasta raiz dentro dele)
pasta_raiz = conteudo_zip[0].split('/')[0]  # Extrai a primeira parte antes da "/"
pasta_extraida_path = os.path.join('/content', pasta_raiz)

# Mostra a pasta extraída
print(f"Pasta extraída: {pasta_extraida_path}")

## 📦🔽 Função para Compactar e Baixar Resultados

Nesta célula, definimos uma função chamada `compactar_e_baixar(pasta_origem)`, que será usada ao final do processamento para facilitar o download dos resultados.

### O que ela faz:
1. **Verifica** se a pasta informada realmente existe.
2. **Compacta a pasta** no formato `.zip`, com o mesmo nome da pasta original.
3. **Inicia automaticamente o download** do arquivo `.zip` para o seu computador, usando a função `files.download()` do Colab.

📁 Essa função será chamada depois que os arquivos de saída forem gerados, tornando o fluxo de trabalho mais prático e sem necessidade de buscas manuais.

In [5]:
import shutil
from google.colab import files
import os

def compactar_e_baixar(pasta_origem):
    if not os.path.isdir(pasta_origem):
        raise ValueError(f"A pasta '{pasta_origem}' não existe.")

    # Gera o caminho do arquivo zip com o mesmo nome da pasta
    nome_zip = os.path.basename(pasta_origem.rstrip('/')) + '.zip'
    caminho_zip = os.path.join('/content', nome_zip)

    # Compacta a pasta
    shutil.make_archive(base_name=caminho_zip.replace('.zip', ''), format='zip', root_dir=pasta_origem)

    # Inicia o download do arquivo zipado
    files.download(caminho_zip)

## 🛠️ Correção de Compatibilidade no Código Fonte

Alguns ambientes, como o Google Colab, não possuem certos módulos disponíveis por padrão. Nesta célula, fazemos uma **modificação automática** no arquivo `Transformer.py` do repositório `bdgd2opendss` para evitar erros de importação.

### O que esta célula faz:
1. Verifica se o arquivo `Transformer.py` existe no caminho esperado.
2. Lê todo o conteúdo do arquivo.
3. **Remove a linha** `from idlelib.pyparse import trans`, que causa erro no Colab por depender de um módulo específico do IDLE.
4. Regrava o arquivo sem essa linha, tornando o código funcional no ambiente atual.

✅ Esse passo garante que o pacote esteja pronto para uso sem necessidade de intervenção manual.

In [ ]:
import os

# Define o caminho completo para o arquivo Transformer.py
file_path = "/content/bdgd2opendss/bdgd2opendss/model/Transformer.py" # Verifique se este caminho está correto após a clonagem

# Verifica se o arquivo existe
if not os.path.exists(file_path):
    print(f"Erro: O arquivo '{file_path}' não foi encontrado. Certifique-se de que o caminho está correto e os arquivos foram carregados no Colab.")
else:
    try:
        # Lê todas as linhas do arquivo
        with open(file_path, "r", encoding="utf-8") as f:
            lines = f.readlines()

        # Filtra a linha indesejada
        # A linha específica a ser removida é 'from idlelib.pyparse import trans'
        filtered_lines = [
            line for line in lines
            if "from idlelib.pyparse import trans" not in line
        ]

        # Escreve as linhas filtradas de volta no arquivo
        with open(file_path, "w", encoding="utf-8") as f:
            f.writelines(filtered_lines)

        print(f"A linha 'from idlelib.pyparse import trans' foi removida de '{file_path}'.")
        print("O arquivo foi modificado com sucesso para ser compatível com o Google Colab.")

    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo '{file_path}': {e}")

## 🚀 Execução do Processamento e Preparação Final dos Resultados

Nesta célula, realizamos as etapas finais para garantir que tudo funcione corretamente no ambiente do Google Colab:

### 🔧 Configurações e Ajustes
- Adicionamos o repositório clonado (`/content/bdgd2opendss`) ao `sys.path`, permitindo a importação de módulos personalizados.
- Garantimos a existência dos arquivos `__init__.py`, que tornam diretórios válidos como pacotes Python.
- Suprimimos *warnings* desnecessários para manter a saída do notebook limpa.

### 📥 Importação da Função Principal
- Importamos a função `run()` do módulo principal, responsável por processar os dados do BDGD e gerar os arquivos desejados.

### ⚙️ Execução do Pipeline
- Definimos os parâmetros de entrada:
  - `bdgd_file_path`: caminho da pasta extraída com os arquivos `.dss`
  - `lst_feeders`: lista com os nomes dos alimentadores a serem processados
  - `output_folder`: pasta onde os resultados serão salvos
- Chamamos a função `run(...)` com os parâmetros definidos.
- Por fim, compactamos a pasta de saída gerada e iniciamos automaticamente o download dos resultados usando a função `compactar_e_baixar`.

✅ **Tudo pronto!** Ao final desta célula, seus dados terão sido processados e você poderá baixar os resultados em formato `.zip`.

In [ ]:
import sys
import pathlib
import os
import warnings

# Define o diretório raiz do repositório clonado
# Certifique-se de que '/content/bdgd2opendss' é o caminho correto onde seu repositório foi clonado.
repo_root_path = '/content/bdgd2opendss'
sys.path.append(repo_root_path)

# Define os caminhos para os arquivos __init__.py que podem estar faltando
package_init_path = os.path.join(repo_root_path, 'bdgd2opendss', '__init__.py')
core_init_path = os.path.join(repo_root_path, 'bdgd2opendss', 'core', '__init__.py')

# Função para criar arquivos __init__.py se não existirem
def create_init_file_if_not_exists(path):
    if not os.path.exists(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            pass # Cria um arquivo vazio
        print(f"Arquivo __init__.py criado em: {path}")
    else:
        print(f"Arquivo __init__.py já existe em: {path}")

# Cria os arquivos __init__.py necessários
create_init_file_if_not_exists(package_init_path)
create_init_file_if_not_exists(core_init_path)


# Suppress RuntimeWarnings globally
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Agora podemos importar a função 'run'
from bdgd2opendss.core.Core import run

if __name__ == '__main__':
    # Em ambientes como o Google Colab ou Jupyter Notebooks, __file__ pode não estar definido.
    # Usamos os.getcwd() para obter o diretório de trabalho atual do notebook.
    script_path = os.getcwd()
    print(f"O diretório atual do script é: {script_path}") # Adicionado para fins de depuração

    bdgd_file_path = pasta_extraida_path
    lst_feeders = ["1325_013_1"]
    output_folder = "/content/"

    # Agora a função 'run' é importada diretamente, então podemos chamá-la sem prefixo
    run(bdgd_file_path=bdgd_file_path, output_folder=output_folder, lst_feeders=lst_feeders, all_feeders=False)

    for feeder in lst_feeders:
      caminho = output_folder + feeder
      compactar_e_baixar(caminho)
